In [1]:
!wget "https://naciscdn.org/naturalearth/110m/physical/ne_110m_ocean.zip"
!wget "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/jovyan/.wget-hsts'. HSTS will be disabled.
--2024-11-06 00:21:23--  https://naciscdn.org/naturalearth/110m/physical/ne_110m_ocean.zip
Resolving naciscdn.org (naciscdn.org)... 99.84.66.70, 99.84.66.57, 99.84.66.2, ...
Connecting to naciscdn.org (naciscdn.org)|99.84.66.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67461 (66K) [application/zip]
Saving to: ‘ne_110m_ocean.zip.1’

ne_110m_ocean.zip.1 100%[===================>]  65.88K  --.-KB/s    in 0.005s  

2024-11-06 00:21:23 (12.0 MB/s) - ‘ne_110m_ocean.zip.1’ saved [67461/67461]

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/jovyan/.wget-hsts'. HSTS will be disabled.
--2024-11-06 00:21:23--  https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip
Resolving naciscdn.org (nacis

In [1]:
import geopandas as gpd
from sedona.spark import *
from pyspark.sql.functions import *
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
countries = gpd.read_file("ne_110m_admin_0_countries.zip")
countries = countries[["ADMIN", "scalerank", "LABELRANK", "ADM0_DIF", "LEVEL", "POP_EST", "POP_RANK", "POP_YEAR", "GDP_MD", "GDP_YEAR", "geometry"]]
countries = sedona.createDataFrame(countries)

In [3]:
target_countries = ["China", "United Kingdom", "Germany", "Vietnam", "South Korea", "Japan", "Portugal", "Australia"]
target_countries = countries.filter(col("ADMIN").isin(target_countries))

In [4]:
target_countries.show()

+--------------+---------+---------+--------+-----+------------+--------+--------+--------+--------+--------------------+
|         ADMIN|scalerank|LABELRANK|ADM0_DIF|LEVEL|     POP_EST|POP_RANK|POP_YEAR|  GDP_MD|GDP_YEAR|            geometry|
+--------------+---------+---------+--------+-----+------------+--------+--------+--------+--------+--------------------+
|       Vietnam|        1|        2|       0|    2| 9.6462106E7|      16|    2019|  261921|    2019|POLYGON ((104.334...|
|   South Korea|        1|        2|       0|    2| 5.1709098E7|      16|    2019| 1646739|    2019|POLYGON ((126.174...|
|       Germany|        1|        2|       0|    2| 8.3132799E7|      16|    2019| 3861123|    2019|POLYGON ((14.1196...|
|      Portugal|        1|        2|       0|    2| 1.0269417E7|      14|    2019|  238785|    2019|POLYGON ((-9.0348...|
|     Australia|        1|        2|       1|    2| 2.5364307E7|      15|    2019| 1396567|    2019|MULTIPOLYGON (((1...|
|         China|        

In [5]:
target_countries_boundary = target_countries.withColumn("buffered_geometry", expr("ST_Buffer(geometry, 75000, true)"))\
    .withColumn("geometry", expr("ST_Difference(buffered_geometry, geometry)"))\
    .select("ADMIN", "geometry", "buffered_geometry")

In [6]:
target_countries_boundary.cache().count()

8

In [7]:
target_countries_boundary.show()

+--------------+--------------------+--------------------+
|         ADMIN|            geometry|   buffered_geometry|
+--------------+--------------------+--------------------+
|       Vietnam|POLYGON ((103.199...|POLYGON ((103.773...|
|   South Korea|POLYGON ((125.543...|POLYGON ((125.607...|
|       Germany|POLYGON ((7.93770...|POLYGON ((8.01116...|
|      Portugal|POLYGON ((-9.7061...|POLYGON ((-9.7053...|
|     Australia|MULTIPOLYGON (((1...|MULTIPOLYGON (((1...|
|         China|POLYGON ((73.1651...|POLYGON ((73.1684...|
|United Kingdom|POLYGON ((-6.3128...|POLYGON ((-6.2688...|
|         Japan|POLYGON ((131.971...|POLYGON ((132.389...|
+--------------+--------------------+--------------------+



We need to clip these buffers to oceans

In [8]:
ocean = gpd.read_file("ne_110m_ocean.zip")

In [9]:
ocean = sedona.createDataFrame(ocean)

In [10]:
target_countries_boundary.createOrReplaceTempView("country")
ocean.createOrReplaceTempView("ocean")

In [11]:
target_countries_boundary.show()

+--------------+--------------------+--------------------+
|         ADMIN|            geometry|   buffered_geometry|
+--------------+--------------------+--------------------+
|       Vietnam|POLYGON ((103.199...|POLYGON ((103.773...|
|   South Korea|POLYGON ((125.543...|POLYGON ((125.607...|
|       Germany|POLYGON ((7.93770...|POLYGON ((8.01116...|
|      Portugal|POLYGON ((-9.7061...|POLYGON ((-9.7053...|
|     Australia|MULTIPOLYGON (((1...|MULTIPOLYGON (((1...|
|         China|POLYGON ((73.1651...|POLYGON ((73.1684...|
|United Kingdom|POLYGON ((-6.3128...|POLYGON ((-6.2688...|
|         Japan|POLYGON ((131.971...|POLYGON ((132.389...|
+--------------+--------------------+--------------------+



In [12]:
target_countries_just_ocean = sedona.sql("""
SELECT c.ADMIN, ST_Intersection(c.buffered_geometry, o.geometry) AS geometry
FROM country c
JOIN ocean o ON ST_Intersects(c.buffered_geometry, o.geometry)  -- Spatial join condition
""")

In [13]:
target_countries_just_ocean.cache().count()
target_countries_just_ocean.show()

+--------------+--------------------+
|         ADMIN|            geometry|
+--------------+--------------------+
|     Australia|MULTIPOLYGON (((1...|
|       Vietnam|POLYGON ((108.714...|
|         China|MULTIPOLYGON (((1...|
|   South Korea|POLYGON ((125.543...|
|         Japan|POLYGON ((131.971...|
|      Portugal|POLYGON ((-9.7061...|
|       Germany|MULTIPOLYGON (((7...|
|United Kingdom|POLYGON ((-6.3128...|
+--------------+--------------------+



In [34]:
#target_countries_just_ocean.repartition(1).write.format("geojson").save("s3://wbts-wbc-m97rcg45xi/77kgrgddb9/data/customer-s539le5sdvtg82/target_countries_just_ocean_single_file.json")

In [16]:
SedonaKepler.create_map(df=target_countries_just_ocean, name="ADMIN")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'ADMIN': {'index': [0, 1, 2, 3, 4, 5, 6, 7], 'columns': ['ADMIN', 'geometry'], 'data': [['Austr…

In [23]:
utm_df = sedona.read.format("geoparquet").load("s3://wbts-wbc-m97rcg45xi/77kgrgddb9/data/customer-s539le5sdvtg82/utm_world.parquet")

In [25]:
SedonaKepler.create_map(df=utm_df)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [26]:
utm_df.createOrReplaceTempView("utm")
target_countries_just_ocean.createOrReplaceTempView("coastal")

In [29]:
utm_df

DataFrame[GZD: string, CM: string, WEST_LONG: string, SOUTH_LAT: string, Zone: string, BandLetter: string, geometry: udt]

In [30]:
target_utm_grids_just_coastal = sedona.sql("""
SELECT c.GZD, c.CM,c.WEST_LONG, c.SOUTH_LAT, c.Zone, c.BandLetter, ST_Intersection(c.geometry, o.geometry) AS geometry
FROM utm c
JOIN coastal o ON ST_Intersects(c.geometry, o.geometry)  -- Spatial join condition
""")

In [31]:
target_utm_grids_just_coastal.cache().count()

58

In [32]:
target_utm_grids_just_coastal.show()

+---+---+---------+---------+----+----------+--------------------+
|GZD| CM|WEST_LONG|SOUTH_LAT|Zone|BandLetter|            geometry|
+---+---+---------+---------+----+----------+--------------------+
|55G|147|      144|      -48|NULL|      NULL|POLYGON ((147.787...|
|50H|117|      114|      -40|NULL|      NULL|POLYGON ((115.081...|
|51H|123|      120|      -40|NULL|      NULL|POLYGON ((119.999...|
|52H|129|      126|      -40|NULL|      NULL|POLYGON ((125.999...|
|53H|135|      132|      -40|NULL|      NULL|POLYGON ((132.000...|
|54H|141|      138|      -40|NULL|      NULL|POLYGON ((137.999...|
|55H|147|      144|      -40|NULL|      NULL|POLYGON ((144 -39...|
|56H|153|      150|      -40|NULL|      NULL|POLYGON ((150.000...|
|49J|111|      108|      -32|NULL|      NULL|POLYGON ((112.784...|
|50J|117|      114|      -32|NULL|      NULL|MULTIPOLYGON (((1...|
|52J|129|      126|      -32|NULL|      NULL|POLYGON ((131.997...|
|53J|135|      132|      -32|NULL|      NULL|POLYGON ((132.711

In [33]:
SedonaKepler.create_map(df=target_utm_grids_just_coastal)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…